Michael Ricardo DS 2500 Project Trading torch

In [ ]:
from datetime import datetime, timedelta
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_ta as ta

In [34]:
def fetch_ticker_data(ticker, years_ago= 5):
    """
    Gets the market data for a given date and ticker.
    Fetches from yfinance library.

    Args:
        years ago (int): Representing the most recent day for stock entries 
        ticker(str): Representing the offical company stock ticker
    Returns
        df (DataFrame): DataFrame with Stock pricing data and history data
    """
    most_recent = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
    five_years_ago = (datetime.today() - timedelta(days= years_ago * 365)).strftime('%Y-%m-%d') 
    
    
    stock_data = yf.Ticker(ticker)
    returning_data = stock_data.history(start = five_years_ago, end= most_recent)
    
    return returning_data

In [35]:
ticker = 'DIS'
dis_data = fetch_ticker_data(ticker)
dis_data = dis_data.drop(columns=['Open', 'High', 'Low', 'Dividends', 'Stock Splits'  ])

Adding variables not related to security itself: 

In [36]:
#Data for the S&P 500 
ticker = '^GSPC'
SP500_data = fetch_ticker_data(ticker)
SP500_data = SP500_data.drop(columns=['Open', 'High', 'Low', 'Dividends', 'Stock Splits'  ])

In [37]:
#CBOE Volatility Index (VIX)
ticker = '^VIX'
VIX_data = fetch_ticker_data(ticker)
VIX_data = VIX_data.drop(columns=['Open', 'High', 'Low', 'Dividends', 'Stock Splits', 'Volume'])

Section will be for data curration and preperation for regression 

In [38]:
def security_editor(df):
    """
    EXPLANATION

    Args:
        
    Returns
        
    """
    df['Returns'] = df['Close'].pct_change() * 100
    df['Next Day Returns'] = df['Returns'].shift(-1)

    return df

In [39]:
dis_refined = security_editor(dis_data)
print(dis_refined)

                                Close    Volume   Returns  Next Day Returns
Date                                                                       
2019-11-27 00:00:00-05:00  149.399445   6155400       NaN          0.066020
2019-11-29 00:00:00-05:00  149.498077   6284900  0.066020         -0.633335
2019-12-02 00:00:00-05:00  148.551254  10351000 -0.633335         -1.354390
2019-12-03 00:00:00-05:00  146.539291   9273800 -1.354390         -0.201924
2019-12-04 00:00:00-05:00  146.243393   7684800 -0.201924         -0.566494
...                               ...       ...       ...               ...
2024-11-18 00:00:00-05:00  113.540001  14236600 -1.338200         -0.986439
2024-11-19 00:00:00-05:00  112.419998  12166800 -0.986439          1.636723
2024-11-20 00:00:00-05:00  114.260002   9319300  1.636723          0.402590
2024-11-21 00:00:00-05:00  114.720001  10261000  0.402590          0.810670
2024-11-22 00:00:00-05:00  115.650002  10093700  0.810670               NaN

[1256 rows 